In [1]:
from FUNCIONES_FEATURES import *
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics as sm
from scipy.stats import pearsonr
from itertools import chain, combinations
import itertools

# DEFINICION

- varianza - 1


- mean_absolute - 2


- median_absolute - 3


- average_range - 4


- coefficient_variation - 5


- quartile_deviation - 6


- mean_deviation - 7


- root_square_deviation - 8


- median_deviation_from_the_mean - 9


- geometric_std - 10


- winsorized_variance - 11


- robust_coefficient_variation - 12

In [2]:
short_window=100
long_window=200

In [3]:
dispersiones = [
    euclidean_distance,
    manhattan_distance,
    minkowski_distance,
    chebyshev_distance,
    bray_curtis_dissimilarity,
    chi_square_distance,
    hellinger_distance,
    earth_movers_distance,
]

In [4]:
def power_set(iterable):
    s = list(iterable)
    return list(
        itertools.chain.from_iterable(
            itertools.combinations(s, r) for r in range(len(s) + 1)
        )
    )

def generacion_score_spread_general(data_trading,data_spread_rate,distance_function):
    dataframes = {}
    for i in distance_function:
        score_ts = scoring(data=data_spread_rate, short_w=short_window, long_w=long_window, distance_function=i)
        spreads_score_df = scoring_vs_max_spread(ts_trading=data_trading, scoring_ts=score_ts, filtracion_n_trades=300)

        dataframes[i] = spreads_score_df
    return dataframes

In [5]:
AL30_TRADING=pd.read_csv("/Users/rogarciag/Desktop/AI_&_TRADING/ALGORITHMIC_TRADING/PRINCIPAL/NASINI/TRABAJOS_GENERAL/PROYECTO_TRADING/BASES_DE_DATOS/DATOS_TRADING_SPREADS/DATOS_SIN_CAUSION/spread_trading_AL30_CAUSION_ESTABLE.csv")
AL30_SPREADS=pd.read_csv('/Users/rogarciag/Desktop/AI_&_TRADING/ALGORITHMIC_TRADING/PRINCIPAL/NASINI/TRABAJOS_GENERAL/PROYECTO_TRADING/BASES_DE_DATOS/DATOS_TRADING_SPREADS/DATOS_SIN_CAUSION/ts_rates_spreads_AL30_CAUSION_ESTABLE.csv')

In [67]:
#AL30_TRADING=AL30_TRADING[0:5000]
#AL30_SPREADS=AL30_SPREADS[0:5000]

In [6]:
dataframes=generacion_score_spread_general(data_trading=AL30_TRADING,data_spread_rate=AL30_SPREADS,distance_function=dispersiones)

In [7]:
euclidean_distance_df=dataframes[euclidean_distance]["Score_Inicio_Trade"]
manhattan_distance_df=dataframes[manhattan_distance]["Score_Inicio_Trade"]
minkowski_distance_df=dataframes[minkowski_distance]["Score_Inicio_Trade"]
chebyshev_distance_df=dataframes[chebyshev_distance]["Score_Inicio_Trade"]
bray_curtis_dissimilarity_df=dataframes[bray_curtis_dissimilarity]["Score_Inicio_Trade"]
chi_square_distance_df=dataframes[chi_square_distance]["Score_Inicio_Trade"]
hellinger_distance_df=dataframes[hellinger_distance]["Score_Inicio_Trade"]
earth_movers_distance_df=dataframes[earth_movers_distance]["Score_Inicio_Trade"]

In [8]:
input_series = [
    euclidean_distance_df,
    manhattan_distance_df,
    minkowski_distance_df,
    chebyshev_distance_df,
    bray_curtis_dissimilarity_df,
    hellinger_distance_df,
    earth_movers_distance_df,
]



indexed_input_series = list(enumerate(input_series, 1))
power_set_series = power_set(indexed_input_series)

sums_df = pd.DataFrame()
for i, combination in enumerate(power_set_series, 1):
    indices = [str(index) for index, _ in combination]
    column_name = "_".join(indices)
    if not column_name:
        column_name = "empty"
    sums_df[f'combination_{column_name}'] = sum(series for _, series in combination)
sums_df["Target"]=dataframes[euclidean_distance]["Maximum Abs Value"]

In [9]:
df=sums_df
df.drop(columns="combination_empty",inplace=True)

# PARAMETRO TRADING -0.06=<   <0.03 

In [61]:
# Assuming your data frame is named df
combination_columns = [col for col in df.columns if 'combination' in col]

r2_scores = {}
correlation_coefficients = {}

for combination_col in combination_columns:
    x = df[combination_col]
    x = np.array(x).reshape(-1, 1)
    y = df['Target']

    sc = StandardScaler().fit(x)
    x_sc = sc.transform(x)

    lm = LinearRegression()
    lm.fit(x_sc, y)
    predictions = lm.predict(x_sc)
    correlation, _ = pearsonr(df[combination_col], df['Target'])

    r2_scores[combination_col] = sm.r2_score(y, predictions)
    correlation_coefficients[combination_col] = correlation

show_top_score=10
sorted_r2_scores = sorted(r2_scores.items(), key=lambda x: x[1], reverse=True)[:show_top_score]
sorted_correlation_coefficients = sorted(correlation_coefficients.items(), key=lambda x: abs(x[1]), reverse=True)[:show_top_score]

print("Top 5 features con mayor r2 positivo:")
for col, r2 in sorted_r2_scores:
    print(f"{col}: {r2:.2f}")

print("\nTop 5 features con mayor correlación absoluta:")
for col, corr in sorted_correlation_coefficients:
    print(f"{col}: {corr:.2f}")


Top 5 features con mayor r2 positivo:
combination_5_6: 0.08
combination_5_6_7: 0.06
combination_6: 0.05
combination_5: 0.05
combination_5_7: 0.04
combination_6_7: 0.04
combination_2_5_6: 0.03
combination_1_2: 0.03
combination_1: 0.03
combination_4_5_6: 0.03

Top 5 features con mayor correlación absoluta:
combination_5_6: 0.28
combination_5_6_7: 0.24
combination_6: 0.23
combination_5: 0.23
combination_5_7: 0.19
combination_6_7: 0.19
combination_2_5_6: 0.18
combination_1_2: -0.18
combination_1: -0.17
combination_4_5_6: 0.17
